# Cars: Getting Started

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import pyblp
sns.set_theme()

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [2]:
cars = pd.read_csv('cars.csv') # this reads the *balanced* dataset (i.e. J = 40 products per market always)
# cars = pd.read_excel('cars.xlsx') # this reads the *unbalanced* dataset (i.e. J varies over time)

In [3]:
lbl_vars = pd.read_csv('labels_variables.csv', index_col=0)
lbl_vals = pd.read_stata('cars.dta', iterator=True).value_labels() # the values that variables take (not relevant for all )

## Overview of the dataset

In [4]:
pd.set_option('display.max_colwidth', None)
tab = cars.mean(numeric_only=True).apply(lambda x: f'{x:.2f}').to_frame('Mean').join(lbl_vars)
tab

,Mean,label
ye,84.50,year (=first dimension of panel)
ma,3.00,market (=second dimension of panel)
co,207.50,model code (=third dimension of panel)
zcode,177.76,alternative model code (predecessors and successors get same number)
brd,16.79,brand code
org,2.72,"origin code (demand side, country with which consumers associate model)"
loc,5.17,"location code (production side, country where producer produce model)"
cla,2.30,class or segment code
home,0.32,domestic car dummy (appropriate interaction of org and ma)
frm,14.50,firm code


# Set up for analysis

## Price variables 

Can be either price (`pr`), price-to-income (`princ`), or log price (`logp`, created below).

In [5]:
price_var = 'eurpr'

In [6]:
cars['logp'] = np.log(cars[price_var])

## Market share

**Todo:** Decide how to measure the market size and thereby the market share. *Note:* Below is just an example that sets the market size = population / 3. 

In [7]:
# total quantity of cars sold in market-year (ma, ye)
cars['qu_tot'] = cars.groupby(['ma', 'ye'])['qu'].transform('sum')
cars['market_size'] = cars['pop'] / 3 # TODO: Choose your own market size measure
cars['s'] = cars['qu'] / cars['market_size']

In [8]:
# compute the share of the outside good (will be useful for the demand inversion)
cars['s0'] = 1.0 - cars.groupby(['ma', 'ye'])['s'].transform('sum')
print(f'Outside share is from {cars.s0.min():.1%} to {cars.s0.max():.1%}')

Outside share is from 88.0% to 95.4%


In [9]:
cars.groupby(['ma'])['s'].describe().rename(index=lbl_vals['market']).style.format('{:.3f}')

,count,mean,std,min,25%,50%,75%,max
ma,,,,,,,,
Belgium,1200.000,0.002,0.001,0.000,0.001,0.002,0.003,0.019
France,1200.000,0.002,0.002,0.000,0.001,0.001,0.003,0.017
Germany,1200.000,0.002,0.003,0.000,0.001,0.001,0.003,0.019
Italy,1200.000,0.002,0.003,0.000,0.001,0.001,0.002,0.023
UK,1200.000,0.002,0.002,0.000,0.001,0.001,0.002,0.010


## 1. Using canned software

In [10]:
from linearmodels.iv import IV2SLS

In [11]:
cars['delta'] = np.log(cars['s'] / cars['s0'])

In [12]:
cars["brand"].replace('alfa romeo', 'alfa_romeo', inplace=True)
cars["brand"] = cars["brand"].str.replace('/', '', regex=False)

In [13]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

In [23]:
# choose your preferred variables 
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li', 'sp', 'ac'] + x_vars_dummies # <--- !!! CHOOSE HERE 
print(x_vars)

['logp', 'home', 'cy', 'hp', 'we', 'li', 'sp', 'ac', 'MCC', 'VW', 'alfa_romeo', 'audi', 'citroen', 'daewoo', 'daf', 'fiat', 'ford', 'honda', 'hyundai', 'innocenti', 'lancia', 'mazda', 'mercedes', 'mitsubishi', 'nissan', 'opel', 'peugeot', 'renault', 'rover', 'saab', 'seat', 'skoda', 'suzuki', 'talbot', 'talhillman', 'talmatra', 'talsimca', 'talsunb', 'toyota', 'volvo']


In [24]:
formula = 'delta ~ 1'
for x_ in x_vars:
    formula += ' + ' + x_
print(formula)
model = IV2SLS.from_formula(formula, cars).fit()

delta ~ 1 + logp + home + cy + hp + we + li + sp + ac + MCC + VW + alfa_romeo + audi + citroen + daewoo + daf + fiat + ford + honda + hyundai + innocenti + lancia + mazda + mercedes + mitsubishi + nissan + opel + peugeot + renault + rover + saab + seat + skoda + suzuki + talbot + talhillman + talmatra + talsimca + talsunb + toyota + volvo


c:\Users\PSand\anaconda3\lib\site-packages\linearmodels\iv\model.py:557: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


ValueError: regressors [exog endog] do not have full column rank

In [18]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:                  delta   R-squared:                      0.3444
Estimator:                        OLS   Adj. R-squared:                 0.3434
No. Observations:                5012   F-statistic:                    2621.5
Date:                Wed, Oct 16 2024   P-value (F-stat)                0.0000
Time:                        15:34:38   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -10.913     0.3505    -31.132     0.0000     -11.600     -10.226
logp           0.2626     0.0403     6.5098     0.0000      0.1836      0.3417
home           0.9578     0.0235     40.722     0.0000      0.9117      1.0039
cy             0.0001  7.617e-05     1.6230     0.1046  -2.566e-05      0.0003
hp            -0.0411     0.0021    -19.689     0.0000     -0.0452     -0.0370
we             0.0005     0.0002     3.4049     0.0007      0.0002      0.0008
li            -0.0110     0.0140    -0.7848     0.4326     -0.0385      0.0165
sp             0.0204     0.0021     9.5752     0.0000      0.0162      0.0245
ac             0.0143     0.0043     3.3567     0.0008      0.0060      0.0227
==============================================================================
"""

## 2. Using numpy

***WARNING:*** The code below works *only* with a *balanced* dataset (i.e. with the same number of products, $J$ for each market (`(ma,ye)` pair.))

### Dummy variables

It can be very important to control for some fixed effects. To do this with matrices, you will have to create dummy variables with one column for each possible value (except one for the reference category). 


### `x_vars`: List of regressors to be used 

In [16]:
K = len(x_vars)
N = cars.ma.nunique() * cars.ye.nunique()
J = 40 
x = cars[x_vars].values.reshape((N,J,K)).astype(float)
cars['outcome'] = cars['s'] / cars['s0']
y = np.log(cars['outcome'].values.reshape((N,J)))

# standardize x
x = ((x - x.mean(0).mean(0))/(x.std(0).std(0)))

# OLS Example

Let's compute the OLS estimator just to test that we can do algebra with the arrays. 

***Note:*** This particular choice of $y$ and $x$ variables might not make sense, it is just to help you get started doing algebra on these arrays. 

In [17]:
Y = y.reshape(N*J,) # Make Y 1-dimensional 
X = np.hstack([x.reshape(N*J,K), np.ones((N*J,1))]) # append a constant term 

In [ ]:
# compute the OLS estimator 
bet = np.linalg.inv(X.T @ X) @ X.T @ Y

# print
varnames = x_vars + ['const'] # we added the constant as the K+1'th column 
pd.DataFrame({'Estimate':bet}, index=varnames)

# Towards non-linear estimation

In order to work with the logit model, you have to be able to compute the utility indices, which typically take the form of some inner product of an $x$-vector and a $\theta$ vector. This is illustrated for you below. Since `x` is `(N,J,K)` (i.e. `x[i,j,:]` gives the $K$-vector of regressors for the car `j` in market-period `i`), we just have to form the matrix product `x @ theta`, and Python will do the sum over the 3rd dimension of `x`. 

In [ ]:
theta0 = np.zeros((K,))
v = x @ theta0 # how to multiply a trial value with the matrix of regressors 
np.exp(v) / np.sum(np.exp(v), 1, keepdims=True) # choice probabilities 